In [1]:
import torch

import flash
from flash.audio import SpeechRecognition, SpeechRecognitionData
from flash.core.data.utils import download_data

In [2]:
# 1. Create the DataModule
download_data("https://pl-flash-data.s3.amazonaws.com/timit_data.zip", "./data")

In [3]:
datamodule = SpeechRecognitionData.from_json(
    "file",
    "text",
    train_file="data/timit/train.json",
    test_file="data/timit/test.json",
    batch_size=4,
)

Using custom data configuration default-bb944d8fb2401c61


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/nm_rostislav/.cache/huggingface/datasets/json/default-bb944d8fb2401c61/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /home/nm_rostislav/.cache/huggingface/datasets/json/default-7c54a7ffbe7b3b49/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

/home/nm_rostislav/.cache/pypoetry/virtualenvs/psm-mFjYNb3v-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3378: FutureWarning: Please pass an instantiated object of the `InputTransform` class. Passing the Class and keyword arguments separately has been deprecated since v0.8.0 and will be removed in v0.9.0.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
datamodule.train_dataset[0]

{'input': array([-2.1362305e-04,  6.1035156e-05,  3.0517578e-05, ...,
        -3.0517578e-05, -9.1552734e-05, -6.1035156e-05], dtype=float32),
 'target': 'Would such an act of refusal be useful?',
 'metadata': {'sampling_rate': 16000}}

In [5]:
# 2. Build the task
model = SpeechRecognition(backbone="facebook/wav2vec2-base-960h")

Using 'facebook/wav2vec2-base-960h' provided by Hugging Face/transformers (https://github.com/huggingface/transformers) and PyTorch/fairseq (https://github.com/pytorch/fairseq).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 3. Create the trainer and finetune the model
trainer = flash.Trainer(max_epochs=10, gpus=1)
trainer.finetune(model, datamodule=datamodule, strategy="freeze")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name          | Type           | Params
-------------------------------------------------
0 | model         | Wav2Vec2ForCTC | 94.4 M
1 | train_metrics | ModuleDict     | 0     
2 | val_metrics   | ModuleDict     | 0     
3 | test_metrics  | ModuleDict     | 0     
-------------------------------------------------
24.6 K    Trainable params
94.4 M    Non-trainable params
94.4 M    Total params
377.585   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
